In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = r'C:\Users\TH\Desktop\NN\cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

def load_data(filename):#从文件里读取数据
    with open(filename,'rb') as f:
        data =  pickle.load(f,encoding='bytes')
        return data[b'data'],data[b'labels']
    
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
            
        self._data = np.vstack(all_data)
        self._data = self._data /127.5 -1 #归一化，很重要，数字太大实现不了
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data
            
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)#混排
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):#返回batch_size个样本
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

    
    
train_filenames = [os.path.join(CIFAR_DIR,'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR,'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

#batch_data,batch_labels = train_data.next_batch(10)
#print(batch_data)
#print(batch_labels)  #测试CifarData类可以正常工作


E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']
(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [2]:
def residual_block(x,output_channel):
    """residual connection implementation"""
    input_channel = x.get_shape().as_list()[-1]#x的最后一维
    if input_channel*2==output_channel:
        increase_dim= True
        strides = (2,2)
    elif input_channel == output_channel:
        increase_dim =False
        strides = (1,1)
    else:
        raise Exception("input channel can't match output channel")
    conv1 = tf.layers.conv2d(x,output_channel,(3,3),strides=strides,padding="same",activation=tf.nn.relu,name='conv1')
    conv2 = tf.layers.conv2d(conv1,output_channel,(3,3),strides=(1,1),padding="same",activation=tf.nn.relu,name='conv2')
    
    if increase_dim:#增加通道数目，做降采样
        pooled_x = tf.layers.average_pooling2d(x,(2,2),(2,2),padding='valid')
        #pooled_x四个维度 [图像数None,image_width,image_weight,channel]->[,,,channel*2]
        padded_x = tf.pad(pooled_x,[[0,0],[0,0],[0,0],[input_channel//2,input_channel//2]])#在最后一个维度增加的数量
    else:
        padded_x = x;
    output_x = conv2+padded_x
    return output_x
        
def res_net(x,num_residual_blocks,num_filter_base,class_num):#num_res_bl:num_subsampling,num_filter:最初的通道数,class_num输出通道数
    """residual network implementation"""
    """
    Args:
    - x:
    - num_residual_blocks: eg: [3, 4, 6, 3]
    - num_filter_base:
    - class_num:
    """
    num_subsampling = len(num_residual_blocks)
    layers = []
    # x: [None, width, height, channel] -> [width, height, channel]
    input_size = x.get_shape().as_list()[1:]#排除掉第一个变量后剩余的size
    with tf.variable_scope('conv0'):
        conv0 = tf.layers.conv2d(x, 
                                 num_filter_base,
                                 (3, 3),
                                 strides = (1, 1),
                                 padding = 'same',
                                 activation = tf.nn.relu,
                                 name = 'conv0')
        layers.append(conv0)
    # eg:num_subsampling = 4, sample_id = [0,1,2,3]
    for sample_id in range(num_subsampling):
        for i in range(num_residual_blocks[sample_id]):
            with tf.variable_scope("conv%d_%d" % (sample_id, i)):
                conv = residual_block(
                    layers[-1],
                    num_filter_base * (2 ** sample_id))#通道数目每次翻倍
                layers.append(conv)
    multiplier = 2 ** (num_subsampling - 1)
    assert layers[-1].get_shape().as_list()[1:] == [input_size[0] / multiplier,
            input_size[1] / multiplier,
            num_filter_base * multiplier]#判断最后的尺寸大小和通道数是否符合预期
    with tf.variable_scope('fc'):
        # layer[-1].shape : [None, width, height, channel]
        # global_pool=kernal_size: image_width, image_height,一张图经过此操作得到一个值——均值
        global_pool = tf.reduce_mean(layers[-1], [1,2])#【1,2】 在1,2维度上做pooling
        logits = tf.layers.dense(global_pool, class_num)
        layers.append(logits)
    return layers[-1]
    
    

x = tf.placeholder(tf.float32,[None,3072])
y = tf.placeholder(tf.int64,[None])    


x_image = tf.reshape(x,[-1,3,32,32])
x_image = tf.transpose(x_image,perm=[0,2,3,1])#交换通道





y_ = res_net(x_image,[2,3,2],32,10)

#交叉熵损失函数
loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
#这个函数可以计算softmax 计算onehot 计算ylogy_

In [3]:
predict = tf.argmax(y_,1)
correct_prediction = tf.equal(predict,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

with tf.Session() as sess:
    init.run()
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,accu_val,_ = sess.run([loss,accuracy,train_op],feed_dict={x:batch_data,y:batch_labels})
        
        if i%500 == 0 :
            print('[Train] step',i,' loss=',loss_val,' acc=',accu_val)
            
        if i%5000==0:
            test_data = CifarData(test_filenames,False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data,test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],feed_dict = {x:test_batch_data,y:test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test step',i+1,' acc=',test_acc)

[Train] step 0  loss= 2.2467482  acc= 0.25
(10000, 3072)
(10000,)
Test step 1  acc= 0.10149999999999998


KeyboardInterrupt: 